# Personal Information
Name: **Ronald Vlaar**

StudentID: **12871524**

Email: [**ronald.vlaar@student.uva.nl**]

Submitted on: **22.03.2024**

TODO:

- Include summary of data (counts etc)
- Show some samples
- Show the segmentation done by ProtoSeg
- Train test split same as protoseg for both datasets
- show IoU for an prediction vs actual using the getIouScoreForLabel in evalPixelLevelSemanticLabeling.py (C:\Users\RonaldVl\.pyenv\pyenv-win\versions\3.10.5\Lib\site-packages\cityscapesscripts\evaluation)

# Data Context
**In this section you should introduce the datasources and datasets which you will be working with. Explain where they are from as well as their domain. Give an overview of what the context of the data is. You should not spend more than 1 to 2 paragraphs here as the core information will be in the next section.**

The first dataset used is the Cityscapes dataset. This dataset can be downloaded from https://www.cityscapes-dataset.com/downloads/. The gtFine_trainvaltest.zip and leftlmgbit_trainvaltest.zip are needed. The domain is image segmentation.


The second dataset is an augmented version of the PASCAL VOC 2012 dataset. The domain is image segmentation. Instruction for downloading the data may be found here https://github.com/kazuto1011/deeplab-pytorch/blob/master/data/datasets/voc12/README.md. The augmented variant (SegmentationAug and SegmentationClassAug) should be downloaded.



The data will be used to train and evaluate a model for image segmentation. We want to improve the ProtoSeg (https://arxiv.org/pdf/2301.12276.pdf) baseline model for image segmentation. This model uses part-prototypes for making its predictions explainable, in contrast too deep learning methods like U-Net and DeepLab.

# Data Description

**Present here the results of your exploratory data analysis. Note that there is no need to have a "story line" - it is more important that you show your understanding of the data and the methods that you will be using in your experiments (i.e. your methodology).**

**As an example, you could show data, label, or group balances, skewness, and basic characterizations of the data. Information about data frequency and distributions as well as results from reduction mechanisms such as PCA could be useful. Furthermore, indicate outliers and how/why you are taking them out of your samples, if you do so.**

**The idea is, that you conduct this analysis to a) understand the data better but b) also to verify the shapes of the distributions and whether they meet the assumptions of the methods that you will attempt to use. Finally, make good use of images, diagrams, and tables to showcase what information you have extracted from your data.**

As you can see, you are in a jupyter notebook environment here. This means that you should focus little on writing text and more on actually exploring your data. If you need to, you can use the amsmath environment in-line: $e=mc^2$ or also in separate equations such as here:

\begin{equation}
    e=mc^2 \mathrm{\space where \space} e,m,c\in \mathbb{R}
\end{equation}

Furthermore, you can insert images such as your data aggregation diagrams like this:

![image](example.png)

In [1]:
# Imports
import os
import numpy as np
import pandas as pd

### Data Loading

In [ ]:
# Load your data here
import cityscapesscripts as cs

### Analysis 1: 
Make sure to add some explanation of what you are doing in your code. This will help you and whoever will read this a lot in following your steps.

In [ ]:
# Also don't forget to comment your code
# This way it's also easier to spot thought errors along the way

### Analysis 2: 

In [ ]:
# ...

### Analysis n:

In [ ]:
# ...